# Creating the OD Matrices
- Baseline: OD matrix with limitation factor lambda (Anastassia)
- Scenario 1: OD matrix  with pop density, deterrence factor and limitation factor lambda (Baseline Yap et al. + Anastassia)
- Scenario 2/3/4: OD matrix with vulnerability  (median income, education level, # jobs within 30min with competition), pop density and deterrence factor lambda (modified Yap et al. + Anastassia)

In [1]:
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'
import osmnx as nx
import shapely
import multiprocess as mp
import numpy as np
import math
import igraph as ig

crs_fr = 2154


In [2]:
#--- Custom function (Anastassia)
%run -i packages.py
def make_attr_dict(*args, **kwargs): 
    
    argCount = len(kwargs)
    
    if argCount > 0:
        attributes = {}
        for kwarg in kwargs:
            attributes[kwarg] = kwargs.get(kwarg, None)
        return attributes
    else:
        return None # (if no attributes are given)

In [3]:
#--- Shapes

# GPM outline
GPM = gpd.read_file('data/raw/GPM.geojson').to_crs(crs_fr)

# IRIS codes and shapes 
IRIS_GPM = gpd.read_file('data/raw/IRIS_GPM.geojson')

## Creating the network and adding igraph IDs to the node table

In [11]:
#--- Create the network
# Retrieve edges
edges_with_id = pd.read_csv('data/clean/initial_network_edges_complete.csv')
edges_with_id["geometry"] = edges_with_id.apply(lambda x: shapely.wkt.loads(x.geometry), axis = 1)
edges_with_id = gpd.GeoDataFrame(edges_with_id, geometry = 'geometry', crs = 4326).to_crs(2154)

# Retrieve nodes
nodes_carbike_centroids_RER_complete = pd.read_csv('data/clean/initial_network_nodes_school_complete.csv')
nodes_carbike_centroids_RER_complete["geometry"] = nodes_carbike_centroids_RER_complete.apply(lambda x: shapely.wkt.loads(x.geometry), axis = 1)
nodes_carbike_centroids_RER_complete = gpd.GeoDataFrame(nodes_carbike_centroids_RER_complete, geometry = 'geometry', crs = 2154)

# Create the attr_dict
nodes_carbike_centroids_RER_complete["attr_dict"] = nodes_carbike_centroids_RER_complete.apply(lambda x: make_attr_dict(
                                                                  nodetype = x.nodetype,
                                                                  centroid = x.centroid,
                                                                  RER = x.RER,
                                                                  IRIS = x.CODE_IRIS,
                                                                  pop_dens = x.pop_density),
                                                                  axis = 1) 

#--- Create Graph with all nodes and edges
G = nx.from_pandas_edgelist(edges_with_id, source='x', target='y', edge_attr=True)
G.add_nodes_from(nodes_carbike_centroids_RER_complete.loc[:,["osmid", "attr_dict"]].itertuples(index = False))

#--- Check if all nodes and edges are present
# Both should be 0
print(len(G.nodes()) - len(nodes_carbike_centroids_RER_complete))
print(len(G.edges()) - len(edges_with_id))

#--- Moving from NetworkX to igraph
g_igraph = ig.Graph()
networkx_graph = G
g_igraph = ig.Graph.from_networkx(networkx_graph)

0
-20


## Creating the basic OD matrix: O, D, lambda

In [12]:
#TODO add lambda thing! 
#TODO add transport mode limitation. you can't walk/bike/walk/bike in alternation 

# Filter nodes for centroids
seq = g_igraph.vs.select(centroid_eq = True)
centroids = [v.index for v in seq]
# Subset if necessary (testing purposes)
nodes = centroids[0:3]

def remove_duplicates(lst):
    unique_lst = []
    for sublist in lst:
        if sorted(sublist) not in [sorted(unique_sublist) for unique_sublist in unique_lst]:
            unique_lst.append(sublist)
    return unique_lst

def process_node(args):
    start_node, end_node = args
    shortest_path_length = g_igraph.shortest_paths_dijkstra(source=start_node, target=end_node, weights='weight')[0][0]
    return (start_node, end_node, shortest_path_length)

if __name__ == '__main__':
    # Number of processes (cores) to use for parallel processing
    num_processes = 4

    # Create a pool of processes
    pool = mp.Pool(processes=num_processes)

    # Generate combinations of nodes for processing
    node_combinations = [(start_node, end_node) for start_node in nodes for end_node in nodes if start_node != end_node]
    node_combinations = remove_duplicates(node_combinations)

    # Apply the function to each node combination using parallel processing
    results = pool.map(process_node, node_combinations)

    # Create a dictionary to store the shortest path lengths
    output = {}
    for start_node, end_node, shortest_path_length in results:
        if start_node not in output:
            output[start_node] = {}
        output[start_node][end_node] = shortest_path_length

    # Create an empty adjacency matrix
    matrix = np.zeros((len(nodes), len(nodes)))

    # Fill the adjacency matrix with shortest path lengths
    for i, start_node in enumerate(nodes):
        for j, end_node in enumerate(nodes):
            if start_node in output and end_node in output[start_node]:
                matrix[i, j] = output[start_node][end_node]
                matrix[j, i] = output[start_node][end_node]

    # Close the pool
    pool.close()
    pool.join()

print(matrix)

/var/folders/2m/lkz691q11b72bpprh6_v34_h0000gn/T/ipykernel_1781/982786006.py:19: DeprecationWarning: Graph.shortest_paths() is deprecated; use Graph.distances() instead
  shortest_path_length = g_igraph.shortest_paths_dijkstra(source=start_node, target=end_node, weights='weight')[0][0]
/var/folders/2m/lkz691q11b72bpprh6_v34_h0000gn/T/ipykernel_1781/982786006.py:19: DeprecationWarning: Graph.shortest_paths() is deprecated; use Graph.distances() instead
  shortest_path_length = g_igraph.shortest_paths_dijkstra(source=start_node, target=end_node, weights='weight')[0][0]
/var/folders/2m/lkz691q11b72bpprh6_v34_h0000gn/T/ipykernel_1781/982786006.py:19: DeprecationWarning: Graph.shortest_paths() is deprecated; use Graph.distances() instead
  shortest_path_length = g_igraph.shortest_paths_dijkstra(source=start_node, target=end_node, weights='weight')[0][0]


[[ 0.         49.74117985 50.25135853]
 [49.74117985  0.         56.34103806]
 [50.25135853 56.34103806  0.        ]]


## Creating the basic OD matrix from Yap et al.: O, D, lambda, population density, deterrence factor

In [13]:
#TODO: do I need lambda if I have a detterence factor?|

In [8]:
#--- Retrieve population density
# TODO adapt to igraph IDs instead of OSMids
# Create a dictionary to store the node attributes
node_pop_density = {}

# Extract the population density from all nodes
for node in nodes:
    pop_density = nodes_carbike_centroids_RER_complete.loc[nodes_carbike_centroids_RER_complete['osmid'] == node]['pop_density'].iloc[0]
    node_pop_density[node] = {'pop_density': pop_density}

IndexError: single positional indexer is out-of-bounds

In [9]:
#--- Calculate OD matrix 
#TODO adapt to igraph IDs

# Create an empty matrix
OD_matrix_original = np.zeros_like(matrix) 

# Find the maximum value of the adjacency matrix (needed for the deterrence factor)
max_value = np.max(matrix)

# Fill in the matrix: (population density i) * (population density j) * exp(- shortest path length / maximum possible shortest path length)
for i in range(len(nodes)):
    for j in range(len(nodes)):
        start_node = nodes[i]
        end_node = nodes[j]
        A = node_pop_density[start_node]['pop_density']
        B = node_pop_density[end_node]['pop_density']
        OD_matrix_original[i, j] = A * B * np.exp(-matrix[i, j] / max_value)

print(OD_matrix_original)

KeyError: 38956

## Create modified OD matrix with economic vulnerability: use median income as the vulnerability attribute 